In [ ]:
def make_verb_upper(text, pos):
    return text.upper() if pos == "VERB" else text

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(u"Cars ran swiftly and walked slowly.")
text = "".join(make_verb_upper(word.text_with_ws, word.pos_) for word in doc)
print(text)
for token in doc:
    print((token.text, token.pos_))

In [ ]:
import pandas as pd

harry_potter = open("HP1.txt").read()
hp = nlp(harry_potter)
hpSents = list(hp.sents)
hpSentsLength = [len(sent) for sent in hpSents]
[sent for sent in hpSents if len(sent)==max(hpSentsLength)]
hpPOS = pd.Series(hp.count_by(spacy.attrs.POS))/len(hp)
tagDict = {w.pos: w.pos_ for w in hp}
hpPos = pd.Series(hp.count_by(spacy.attrs.POS))/len(hp)
df = pd.DataFrame([hpPOS], index=["Harry Potter"])
df.columns = [tagDict[column] for column in df.columns] 
df.T.plot(kind="bar")

In [ ]:
from collections import Counter
hpAdjs = [w for w in hp if w.pos_ == "PRON"]
Counter([w.text.strip() for w in hpAdjs]).most_common(10)

In [ ]:
sent_0 = nlp(u"Donald Trump visited cars at the government headquarters in France today.")
for token in sent_0:
    print(token.text, token.ent_type_)
    
for ent in sent_0.ents:
    print(ent.text, ent.label_)

A pos tagger to recognize the entity "USTHB" and "LRIA" as ORG

In [1]:
from spacy.training.example import Example
import random 
import spacy

In [2]:
TRAIN_DATA = [
    ("Cars are great. They're fast.", {"tags": ["NSS", "VERB", "ADJ", "PUNCT", "PRON", "VERB", "ADV", "PUNCT"]}),
    ("Cats and dogs are pets.", {"tags": ["NSS", "CCONJ", "NSS", "VERB", "NOUN", "PUNCT"]}),
    ("The doors of the house are red.", {"tags": ["DET", "NSS", "ADP", "DET", "NOUN", "VERB", "ADJ", "PUNCT"]}),
    ("I have pillows on my bed.", {"tags": ["PRON", "VERB", "NSS", "ADP", "PRON", "NOUN", "PUNCT"]}),
    ("The kids play with toys.", {"tags": ["DET", "NSS", "VERB", "ADP", "NOUN", "PUNCT"]}),
    ("These books belong to the library.", {"tags": ["DET", "NSS", "VERB", "ADP", "DET", "NOUN", "PUNCT"]}),
    ("The girls and boys are playing.", {"tags": ["DET", "NSS", "CCONJ", "NSS", "VERB", "VERB", "PUNCT"]}),
    ("They brought pizzas for us.", {"tags": ["PRON", "VERB", "NSS", "ADP", "PRON", "PUNCT"]}),
    ("The windows in the house are large.", {"tags": ["DET", "NSS", "ADP", "DET", "NOUN", "VERB", "ADJ", "PUNCT"]}),
    ("The trees and flowers look beautiful.", {"tags": ["DET", "NSS", "CCONJ", "NSS", "VERB", "ADJ", "PUNCT"]}),
    ("These chairs are comfortable.", {"tags": ["DET", "NSS", "VERB", "ADJ", "PUNCT"]}),
    ("She bought dresses for the party.", {"tags": ["PRON", "VERB", "NSS", "ADP", "DET", "NOUN", "PUNCT"]}),
    ("The workers are on strike.", {"tags": ["DET", "NOUN", "VERB", "ADP", "NOUN", "PUNCT"]}),
    ("The apples and oranges are juicy.", {"tags": ["DET", "NSS", "CCONJ", "NSS", "VERB", "ADJ", "PUNCT"]}),
    ("We need pencils and erasers.", {"tags": ["PRON", "VERB", "NSS", "CCONJ", "NSS", "PUNCT"]}),
    ("These cars have good mileage.", {"tags": ["DET", "NSS", "VERB", "ADJ", "NOUN", "PUNCT"]}),
    ("The hats and scarves are in the closet.", {"tags": ["DET", "NSS", "CCONJ", "NSS", "VERB", "ADP", "DET", "NOUN", "PUNCT"]}),
    ("She collects stamps and coins.", {"tags": ["PRON", "VERB", "NSS", "CCONJ", "NSS", "PUNCT"]}),
    ("The students passed their exams.", {"tags": ["DET", "NOUN", "VERB", "DET", "NOUN", "PUNCT"]}),
    ("The doors and windows are locked.", {"tags": ["DET", "NSS", "CCONJ", "NSS", "VERB", "VERB", "PUNCT"]}),
]

In [3]:
nlp = spacy.blank("en")
pos_tagger = nlp.add_pipe("tagger")

tags=["AUX", "PUNCT", "PROPN", "ADJ", "ADP", "NSS", "VERB", "PART", "DET", "NOUN", "ADV","CCONJ", "PRON","NUM"]
for tag in tags:
    pos_tagger.add_label(tag)
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'tagger']

In [4]:
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for i in range(10):
        random.shuffle(TRAIN_DATA)
        for text, annotation in TRAIN_DATA:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp.update([example], drop=0.5, losses={})

In [6]:
doc = nlp("Cats are great. They're fast.")
for token in doc:
    print(token.text, token.tag_)

Cats NSS
are VERB
great ADJ
. PUNCT
They PRON
're VERB
fast ADJ
. PUNCT


A ner tagger to recognize new entity label called "COLOR"

In [1]:
import random
import spacy
from spacy.training import Example

In [5]:
TRAIN_DATA2 = [
    ("The sky is blue.", {"entities": [(11, 15, "COLOR")]}),
    ("She is wearing a beautiful red dress.", {"entities": [(27, 30, "COLOR")]}),
    ("The grass in the park is a vibrant green.", {"entities": [(35, 40, "COLOR")]}),
    ("The sunset painted the sky with shades of orange and pink.", {"entities": [(42, 48, "COLOR"), (53, 57, "COLOR")]}),
    ("The car was a sleek black sports model.", {"entities": [(20, 25, "COLOR")]}),
    ("The flowers in the garden are various shades of purple.", {"entities": [(48, 54, "COLOR")]}),
    ("The ocean water was a deep, dark blue.", {"entities": [(28, 37, "COLOR")]}),
    ("The leaves in the fall turn a beautiful golden color.", {"entities": [(40, 46, "COLOR")]}),
    ("Her hair is a natural shade of brown.", {"entities": [(31, 36, "COLOR")]}),
    ("The coffee in the cup was a rich, mahogany brown.", {"entities": [(43, 48, "COLOR")]}),
    ("The curtains in the room are a soft shade of beige.", {"entities": [(45, 50, "COLOR")]}),
    ("The car's paint was a shiny metallic silver.", {"entities": [(37, 43, "COLOR")]}),
    ("The roses in the garden are a deep, velvety red.", {"entities": [(44, 47, "COLOR")]}),
    ("The umbrella was a cheerful shade of yellow.", {"entities": [(37, 43, "COLOR")]}),
    ("His eyes sparkled with a bright emerald green.", {"entities": [(40, 45, "COLOR")]}),
    ("The room was painted a soothing pale blue.", {"entities": [(37, 41, "COLOR")]}),
    ("The butterfly had wings with intricate patterns of black and orange.", {"entities": [(51, 56, "COLOR"), (61, 67, "COLOR")]}),
    ("The sea is a stunning azure blue.", {"entities": [(28, 32, "COLOR")]}),
    ("The car's interior is a luxurious shade of cream.", {"entities": [(43, 48, "COLOR")]}),
    ("The ripe strawberries are a deep crimson red.", {"entities": [(41, 44, "COLOR")]}),
]


In [3]:
nlp_ner = spacy.load("en_core_web_sm")
ner_tagger = nlp_ner.get_pipe("ner")

tags=["COLOR"]
for tag in tags:
    ner_tagger.add_label(tag)
other_pipes = [pipe for pipe in nlp_ner.pipe_names if pipe != 'ner']
with nlp_ner.disable_pipes(*other_pipes):
    optimizer2 = nlp_ner.create_optimizer()
    for i in range(10):
        random.shuffle(TRAIN_DATA2)
        for text, annotation in TRAIN_DATA2:
            doc = nlp_ner.make_doc(text)
            example = Example.from_dict(doc, annotation)
            nlp_ner.update([example], sgd=optimizer2)


In [6]:
doc_ner = nlp_ner("The grass is green")
print(doc_ner)
for ent in doc_ner.ents:
    print((ent.text, ent.label_))

The grass is green
('green', 'COLOR')
